# 🚶‍♂️ The Validation, by walking in the forward

The "*walk forward - validation*" is the *technique* that will let us fit the model **which has generalized** the overall pattern in the data and is not *overfitted*.

> But... we do have train-test-split right?

## The problems

## 1️⃣
Till now, I have used the old school `ttsplit` technique to do the work... but that **again** *("again" because that same problem rises in the common ML scenario)* creates problems because we are only partitionling the data in `2` parts and while testing, with the test data, we *tune the model* by ourselves — and hence the testing data becomes the "in-sample" data ie. the model might not have been generalized yet — only optimized for the training and testing data.

## 2️⃣
The solution was **to use the Cross-Validation**. The shiny, glittering, jargon, used by professors and paper readers term: "Cross. Vali. Dation."

Yeah, I mean literally this technique is like, when I read this... I've got it! But when it comes in different context... *I might wanna re-check it once*. Coming back to the problems: Can we use the Cross validation in the time series setting?

> Of course... not!

**Why?** <br>
The reason is — the CV split *"kind of randomized data"*. While the time-series being a sequential data, ***we can't actually let it randomize it***. Consider this → 1, 2, 3, 4, 5 are the months in 2022. For 1st CV it will use 2, 3, 4, 5 as training and 1 for valucation. Fine, but then it will use 1, 3, 4, 5 and 2 for validation. There is a gap man!

And also, the data points inside can be randomized as well!

<img src="../images/no-nooo.gif" hight=300 width=200>

Okay got it... but what is the freakin' solution? How can I save my model not to **mug up** the data?

### 

## 📝 The WalkForward Validation
*Notice, there is no "CROSS"*.

<img src="../images/walk-forward-validation.png" height=300 width=500>

- We will start with *some minimum* amount of data.
- Then we will forecast the `h` points.
- That means: we will only take `h` sized validation set.
- Then we will move some `offest` step ahead and repeat the same *(offset say, 1)*.
- This will continue **till we reach to end** of the data.


    `data`: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11
    `h`: 3
    
    Step 1 — `train-set`: 1 | `validation-set`: 2, 3, 4
    Step 1.1 — Train → forecast `h` and validate 
    Step 2 — `train-set`: 1, 2 | `validation-set`: 3, 4, 5
    .
    .
    .
    Step n - hth: — `train-set`: 1, 2, 3, 4, 5, 6, 7, 8 | `validation-set`: 9, 10, 11
    (here 8th)

😊 માજા આવી?

###### 

### And ausaau
We can choose whether or not to take "full data" or not. Meaning, if in our case the data relies more on the recent past data, and we also want the *recent* data to be in the training... ***and instead of growing*** the data continuously by some `offset` term... we can keep the **datasize *constant***.

In [5]:
import IPython
IPython.display.Audio("../images/auso.mp3")